In [ ]:
# ==============================================================
# Ultra-Robust Ensemble Regime Classification Pipeline
# ==============================================================

import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import (
    VotingClassifier,
    RandomForestClassifier,
    ExtraTreesClassifier
)
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (
    classification_report,
    accuracy_score,
    ConfusionMatrixDisplay,
    roc_auc_score,
    f1_score
)
from scipy.stats import randint, uniform, loguniform

# ==============================================================
# 1. Load and Prepare Data
# ==============================================================

train_df = pd.read_csv("../data/generated_data/train.csv")
test_df = pd.read_csv("../data/generated_data/test.csv")

drop_cols = ["date"]
train_df = train_df.select_dtypes(include=np.number).drop(columns=drop_cols, errors="ignore")
test_df = test_df.select_dtypes(include=np.number).drop(columns=drop_cols, errors="ignore")

train_df["regime_t+3"] = train_df["regime"].shift(-3)
test_df["regime_t+3"] = test_df["regime"].shift(-3)

train_df = train_df.drop(columns=["regime"]).dropna(subset=["regime_t+3"])
test_df = test_df.drop(columns=["regime"]).dropna(subset=["regime_t+3"])

X_train = train_df.drop(columns=["regime_t+3"])
y_train = train_df["regime_t+3"].astype(int) - 1
X_test = test_df.drop(columns=["regime_t+3"])
y_test = test_df["regime_t+3"].astype(int) - 1

scaler = RobustScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ==============================================================
# 2. Define All Base Models
# ==============================================================

xgb = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42)
rf = RandomForestClassifier(random_state=42)
et = ExtraTreesClassifier(random_state=42)
mlp = MLPClassifier(max_iter=10000, random_state=42)
svc = SVC(probability=True, random_state=42)
log_reg = LogisticRegression(max_iter=10000, random_state=42)
lda = LinearDiscriminantAnalysis()
knn = KNeighborsClassifier()

# ==============================================================
# 3. Ensemble Definition
# ==============================================================

voting_clf = VotingClassifier(
    estimators=[
        ("xgb", xgb),
        ("rf", rf),
        ("et", et),
        ("mlp", mlp),
        ("svc", svc),
        ("log_reg", log_reg),
        ("lda", lda),
        ("knn", knn),
    ],
    voting="soft",
    n_jobs=-1
)

pipeline = Pipeline([
    ("clf", voting_clf)
])

# ==============================================================
# 4. Massive Randomized Hyperparameter Search
# ==============================================================

param_distributions = {
    # XGBoost
    "clf__xgb__n_estimators": randint(300, 1200),
    "clf__xgb__max_depth": randint(3, 12),
    "clf__xgb__learning_rate": loguniform(1e-3, 0.3),
    "clf__xgb__subsample": uniform(0.5, 0.5),
    "clf__xgb__colsample_bytree": uniform(0.5, 0.5),
    "clf__xgb__gamma": uniform(0, 5),

    # Random Forest
    "clf__rf__n_estimators": randint(300, 1200),
    "clf__rf__max_depth": [None] + list(range(5, 40, 5)),
    "clf__rf__min_samples_leaf": randint(1, 10),

    # Extra Trees
    "clf__et__n_estimators": randint(300, 1200),
    "clf__et__max_depth": [None] + list(range(5, 40, 5)),
    "clf__et__min_samples_leaf": randint(1, 10),

    # MLP
    "clf__mlp__hidden_layer_sizes": [(64,), (128,), (64, 32), (128, 64), (256, 128), (512, 256)],
    "clf__mlp__activation": ["relu", "tanh"],
    "clf__mlp__alpha": loguniform(1e-6, 1e-2),
    "clf__mlp__learning_rate_init": loguniform(1e-4, 1e-2),

    # SVC
    "clf__svc__C": loguniform(1e-2, 10),
    "clf__svc__gamma": loguniform(1e-4, 1e-1),
    "clf__svc__kernel": ["rbf", "poly", "sigmoid"],

    # Logistic Regression
    "clf__log_reg__C": loguniform(1e-3, 10),
    "clf__log_reg__penalty": ["l2"],

    # KNN
    "clf__knn__n_neighbors": randint(3, 20),
    "clf__knn__weights": ["uniform", "distance"],
    "clf__knn__p": [1, 2],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=300,  # 🔥 Expand this for deeper search
    scoring="roc_auc_ovr",
    cv=cv,
    n_jobs=-1,
    verbose=3,
    random_state=42
)

search.fit(X_train_scaled, y_train)

print("\n== Best Ensemble Parameters ==")
print(search.best_params_)
print(f"Best CV AUC (OvR): {search.best_score_:.4f}")

best_model = search.best_estimator_

# ==============================================================
# 5. Massive Weight Optimization Search
# ==============================================================

estimators_list = [
    "xgb", "rf", "et", "mlp", "svc", "log_reg", "lda", "knn"
]

weight_combos = list(itertools.product([1, 2, 3, 4, 5], repeat=len(estimators_list)))
print(f"Total weight combinations: {len(weight_combos)}")

best_auc, best_f1, best_weights = -np.inf, -np.inf, None

for w in weight_combos:
    weighted_clf = VotingClassifier(
        estimators=[(name, best_model.named_steps["clf"].named_estimators_[name]) for name in estimators_list],
        voting="soft",
        weights=w,
        n_jobs=-1
    )

    weighted_clf.fit(X_train_scaled, y_train)
    y_pred_proba = weighted_clf.predict_proba(X_test_scaled)
    y_pred = np.argmax(y_pred_proba, axis=1)

    auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
    f1 = f1_score(y_test, y_pred, average="weighted")
    score = (auc + f1) / 2  # Combined metric

    if score > best_auc:
        best_auc, best_weights, best_f1 = score, w, f1

print("\n== Best Weights Found ==")
print(f"Weights: {best_weights}\nCombined Score: {best_auc:.4f}, F1: {best_f1:.4f}")

final_model = weighted_clf

# ==============================================================
# 6. Evaluate on Test Set
# ==============================================================

y_pred_test = final_model.predict(X_test_scaled)
y_test_proba = final_model.predict_proba(X_test_scaled)

print("\n== Ensemble — TEST Accuracy Report ==")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(classification_report(y_test, y_pred_test, digits=4))

ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_test,
    cmap="Blues",
    normalize="true",
    values_format=".2f"
)
plt.title("Ensemble — Normalized Test Confusion Matrix")
plt.tight_layout()
plt.show()

auc_score = roc_auc_score(y_test, y_test_proba, multi_class="ovr")
print(f"Test Set AUC (OvR): {auc_score:.4f}")

# ==============================================================
# 7. Confidence Decile Analysis
# ==============================================================

confidences = y_test_proba.max(axis=1)
y_pred = np.argmax(y_test_proba, axis=1)

eval_df = pd.DataFrame({
    "true": y_test + 1,
    "pred": y_pred + 1,
    "conf": confidences
})
eval_df["correct"] = (eval_df["true"] == eval_df["pred"]).astype(int)
eval_df = eval_df.sort_values("conf", ascending=False).reset_index(drop=True)

decile_results = []
n = len(eval_df)
for d in range(5, 105, 5):
    cutoff = int((d / 100) * n)
    subset = eval_df.iloc[:cutoff]
    accuracy = subset["correct"].mean()
    avg_conf = subset["conf"].mean()
    decile_results.append({
        "Top %": f"Top {d}%",
        "Accuracy": round(accuracy, 3),
        "Avg Confidence": round(avg_conf, 3)
    })

decile_df = pd.DataFrame(decile_results)
print("\n== Confidence Decile Table ==")
print(decile_df)


In [ ]:
# ==========================================
# COMPARE MODEL ACCURACY VS BASELINE FREQUENCY
# ==========================================
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# --- Example baseline distribution (replace with yours dynamically) ---
baseline_dist = pd.DataFrame({
    "regime": [
        "UpMild_LowVol", "DownMild_LowVol", "Neutral_LowVol", "DownHard_HighVol",
        "UpMild_HighVol", "DownMild_HighVol", "UpHard_HighVol", "UpHard_LowVol",
        "Neutral_HighVol", "DownHard_LowVol"
    ],
    "Percent": [24.03, 15.64, 14.86, 11.90, 7.91, 7.37, 5.80, 4.77, 4.17, 3.56]
})

# --- Compute model accuracy per class ---
from sklearn.metrics import accuracy_score

# Assuming you have y_test and y_pred_test from your final model
model_acc = pd.crosstab(y_test, y_pred_test, normalize='index')

# Per-class accuracy: diagonal of confusion matrix
class_acc = np.diag(model_acc.values)

# Map integer labels back to regime names (if needed)
if len(baseline_dist) == len(class_acc):
    model_perf = pd.DataFrame({
        "regime": baseline_dist["regime"],
        "Model_Accuracy": class_acc * 100  # convert to %
    })
else:
    raise ValueError("Mismatch between baseline regime count and model outputs")

# --- Combine for plotting ---
compare_df = baseline_dist.merge(model_perf, on="regime", how="inner").sort_values("Percent", ascending=False)

# --- Plot ---
plt.figure(figsize=(10,8))
bar_width = 0.35
x = np.arange(len(compare_df))

plt.bar(x - bar_width/2, compare_df["Percent"], width=bar_width, label="Baseline Frequency (%)", color="lightgray")
plt.bar(x + bar_width/2, compare_df["Model_Accuracy"], width=bar_width, label="Model Accuracy (%)", color="steelblue")

plt.xticks(x, compare_df["regime"], rotation=30, ha="right")
plt.ylabel("Percentage", fontsize=12)
plt.title("Baseline Distribution vs Model Per-Class Accuracy", fontsize=16, weight="bold")
plt.legend()
plt.tight_layout()
plt.show()
